# Convolution Neural Network - Example of Cats and Dog Classification

In [1]:
# Importing Libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# If we want to read the data from folders - ImageDataGenerator from Keras will help us to do this.

In [2]:
# tensorflow version

tf.__version__

'2.2.0'

# Part 1 - Data Preprocessing

In [9]:
# Preprocessing the training set

train_datagen = ImageDataGenerator(rescale = 1.0/255,  # Normalizing the data. Bringing the value between 0 and 1.
                                                       # We are dividing by 255 because, the image have the color range
                                                       # from 0 to 255. So, dividing each pixel of an image by 255 will 
                                                       # bring the value between 0 and 1 
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# ImageDataGenerator will also allow us to do feature scaling on images. So that we will have some more new images. 
# This is called Data Augmentation.

# Note: 
# Only rescaling operation, we need to apply to our test dataset. Remaining things mentioned in the above no need to apply.

In [10]:
# Need to read the data from the folder

# Preprocessing the training set
train_set = train_datagen.flow_from_directory('Datasets/train',
                                               target_size =  (64,64), # image size
                                               batch_size = 32,   # In each epoch 32 iterations will be done
                                               class_mode = 'binary') # We are doing class and Dog i.e, 0 or 1.
                                                # If we dont have binary classification problem, in that case we specify
                                                # class_mode = 'categorical'

# Preprocessing the test set
test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_set = test_datagen.flow_from_directory('Datasets/train',
                                             target_size =  (64,64),
                                             batch_size = 32,
                                             class_mode = 'binary')


# The Target size should be same, when constructing the Neural Network.
# Note:
# If the image size is bigger we can make it as a smaller image, by specifying in the target_size
# If the image size is smaller, we can make it as a bigger image. But, we wont get clarity. so avoid giving target_size bigger 
# value compared to actual size of the image

Found 2002 images belonging to 2 classes.
Found 2002 images belonging to 2 classes.


# Part 2 - Building CNN

In [11]:
# Intializing the Sequenatial model. - Which will help us to do forward and backward propogation
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu', input_shape=[64,64,3]))
# For each of pur 32 filter kernel size is 3x3.
# Input size is 64x64 as we mentioned in train_Set and 3 is RGB image
# padding='same'- After every filter operation, it will ensure that output image size is same.
# If we not specify strides, then it will taken as 1.

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding second convolution layer
cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
# Units is number of hidden neurons.
# We can any number of dense layer

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Since it is a binary classification, we using sigmoid as an activation function

In [12]:
# Summary of our CNN  model
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1048704   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

# Part3 - Training our CNN Model

In [21]:
# Compiling the CNN
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model on the Training set and evaluating on the test set
cnn.fit(x= train_set, validation_data = test_set, epochs=25)

Epoch 1/25
63/63 [==============================] - 19s 307ms/step - loss: 0.6152 - accuracy: 0.6688 - val_loss: 0.5956 - val_accuracy: 0.6843
Epoch 2/25
63/63 [==============================] - 20s 314ms/step - loss: 0.5943 - accuracy: 0.6893 - val_loss: 0.5729 - val_accuracy: 0.7018
Epoch 3/25
63/63 [==============================] - 20s 313ms/step - loss: 0.5692 - accuracy: 0.7103 - val_loss: 0.5531 - val_accuracy: 0.7198
Epoch 4/25
63/63 [==============================] - 19s 309ms/step - loss: 0.5485 - accuracy: 0.7173 - val_loss: 0.5404 - val_accuracy: 0.7123
Epoch 5/25
63/63 [==============================] - 19s 304ms/step - loss: 0.5274 - accuracy: 0.7473 - val_loss: 0.5370 - val_accuracy: 0.7153
Epoch 6/25
63/63 [==============================] - 20s 315ms/step - loss: 0.5173 - accuracy: 0.7547 - val_loss: 0.5873 - val_accuracy: 0.6848
Epoch 7/25
63/63 [==============================] - 19s 300ms/step - loss: 0.5082 - accuracy: 0.7502 - val_loss: 0.4507 - val_accuracy: 0.7977

# Part4 - Saving a Model

In [22]:
cnn.save('dog_cat_classification_model.h5')

# Part5 - Loading the save model and test

In [24]:
from tensorflow.keras.models import load_model

model = load_model('dog_cat_classification_model.h5')

In [29]:
# Making predictions from the load model

import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('Datasets/test/Dog/10003.jpg', target_size=[64,64,3])
test_image = image.img_to_array(test_image)
test_image = test_image/255
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)

result

# Steps:
# Loading the image and give the targetsize same as we mentioned in the train set and test set
# Conveting it to an array
# Dividing by 255 because we rescaling in the test set
# expand the dims
# predict the image

array([[0.9766883]], dtype=float32)

In [30]:
if result[0] <= 0.5:
    print("The image is a Cat")
else:
    print("The image is a Dog")
    
# We using [0] for Cat because, in Dataset folder, Cat folder will be first and Dog folder will be second 
# because of alphabetical order

The image is a Dog


# Issues while Training

In [31]:
# If you face any error like below:
# PIL.UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000002121AC9D8E0>

# This issue is due to: There was an issue with one of the img that was causing an issue
# To solve the above issue, run the below command and find the incorrect image 
# and replace that image with the new image with the same label name as mentioned previously in that image.

In [19]:
import PIL
from pathlib import Path
from PIL import UnidentifiedImageError

path = Path("Datasets/train/Cat").rglob("*.jpg")
for img_p in path:
    try:
        img = PIL.Image.open(img_p)
    except PIL.UnidentifiedImageError:
            print(img_p)